In [1]:
import pandas as pd
import sqlite3

In [2]:
conn=sqlite3.connect('database.db')

In [3]:
df = pd.read_sql_query("SELECT * FROM crime_data", conn)

In [4]:
df

,Id,IncidentNum,IncidentDate,TimeOccurred,SLMPDOffense,NIBRSCode,NIBRSCat,NIBRSOffenseType,UCR_SRS,CrimeGrade,...,District,Neighborhood,NeighborhoodNum,Latitude,Longitude,Supplemented,SupplementDate,VictimNum,FirearmUsed,IncidentNature
0,0,21000002,1/1/2021,02:00.0,MISCELLANEOUS WEAPON VIOLATION,520,None,None,None,None,...,4.0,36,None,38.629573,-90.204995,None,None,None,None,None
1,1,21000003,1/1/2021,09:00.0,MISCELLANEOUS WEAPON VIOLATION,520,None,None,None,None,...,5.0,38,None,38.641221,-90.264226,None,None,None,None,None
2,2,21000003,1/1/2021,09:00.0,PROPERTY DAMAGE,290,None,None,None,None,...,5.0,38,None,38.641221,-90.264226,None,None,None,None,None
3,3,21000007,1/1/2021,10:00.0,PROPERTY DAMAGE - 2ND DEGREE,290,None,None,None,None,...,2.0,9,None,38.598725,-90.310722,None,None,None,None,None
4,4,21000008,1/1/2021,20:00.0,MISCELLANEOUS WEAPON VIOLATION,520,None,None,None,None,...,1.0,16,None,38.579494,-90.243210,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173359,173359,24000551,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),290,None,None,None,None,...,1.0,16,None,38.587915,-90.252870,None,None,None,None,None
173360,173360,24000557,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),290,None,None,None,None,...,6.0,68,None,38.677708,-90.217160,None,None,None,None,None
173361,173361,24000591,12/31/2023,00:00.0,DISCHARGING FIREARM WITHIN CITY,520,None,None,None,None,...,1.0,3,None,38.569796,-90.254490,None,None,None,None,None
173362,173362,24000629,12/31/2023,00:00.0,STEALING UNDER $150 (PETTY LARCENY-MOTOR VEH P...,23G,None,None,None,None,...,4.0,36,None,38.630687,-90.201630,None,None,None,None,None


In [ ]:
df.info()

In [ ]:
conn.close()

In [ ]:
df = pd.read_csv('uploads/Crime2021-2023.csv')
df